In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data Prepartion

In [7]:
df = pd.read_excel('Online Retail.xlsx')
df.sample(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
134844,547870,22178,VICTORIAN GLASS HANGING T-LIGHT,6,2011-03-27 14:18:00,1.25,13668.0,United Kingdom
350919,567656,84968D,SET OF 16 VINTAGE RED CUTLERY,1,2011-09-21 14:40:00,24.96,14096.0,United Kingdom
401965,571456,23148,MINIATURE ANTIQUE ROSE HOOK IVORY,11,2011-10-17 13:57:00,0.83,16729.0,United Kingdom


In [8]:
df.shape

(541909, 8)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [14]:
df.nunique() #Tells the unique values 

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

In [15]:
df['Country'].value_counts()

#Majority of the data is from UK

United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

Since majority of the data is from UK we will be performing the analysis on them. We also won't care about cancelled orders and drop all the values. 

In [61]:
df_uk = df[df['Country']=='United Kingdom']
df_uk.shape

#We are selecting only United Kingdom (UK) data

(495478, 8)

In [62]:
df_uk = df_uk[df_uk['Quantity']>0] 
df_uk.shape
#We want want where is positive quantity as negative represents returned items

(486286, 8)

In [63]:
df_uk.dropna(subset=['CustomerID'], how = 'all', axis = 0, inplace = True)
df_uk.shape
#We are dropping all the NAN values where there is no customerID 

(354345, 8)

In [64]:
#Typically RFM analysis is better if done for a particular period so let's restrict to one year

df_uk[df_uk['InvoiceDate']<'2011'].count()

InvoiceNo      23943
StockCode      23943
Description    23943
Quantity       23943
InvoiceDate    23943
UnitPrice      23943
CustomerID     23943
Country        23943
dtype: int64

In [65]:
df_uk[df_uk['InvoiceDate']>'2011'].count()

InvoiceNo      330402
StockCode      330402
Description    330402
Quantity       330402
InvoiceDate    330402
UnitPrice      330402
CustomerID     330402
Country        330402
dtype: int64

In [66]:
df_uk[df_uk['InvoiceDate']>'2012'].count()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [69]:
# Since 2011 has highest records we will be using that

df_uk = df_uk[df_uk['InvoiceDate']>='2010-12-09']
df_uk.shape

(342478, 8)

In [70]:
#Understand the data

df_uk.nunique()

InvoiceNo      16017
StockCode       3611
Description     3799
Quantity         289
InvoiceDate    15040
UnitPrice        396
CustomerID      3863
Country            1
dtype: int64

* We have a total of 16,017 unique transactions
* We have a 3611 of unique products
* We have 3663 of total customes

# RFM Analysis

RFM (Recency, Frequency, Monetary) analysis is a customer segmentation techniqueused t o identify a company's best customers by using certain measures. We use past purchase behavior of customers and divide them into groups. After we divide customers into various categories or clusters we can identify customers who are more likely to respond to promotions.

Recency (R) tells us days since last purchase, normally lower the R the better.

Frequency (F) tells us total number of purchases by the customer so higher the frequecy the better.

Monetary Value (M) tells us total amount spent by that customer, so higher the better.

Let's create these features and let's find out the where the best customers lie in our dataset.



### Recency (R)

Since we have restricted the data to 2011, we will start from the last date used in the year 

In [74]:
import datetime as dt

In [75]:
now = dt.date(2011,12,31)
print(now)

2011-12-31


In [76]:
df_uk['lastdate'] = df_uk['InvoiceDate'].dt.date
df_uk.head(3)

#We are creating a feature with only the date

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,lastdate
19632,537879,22114,HOT WATER BOTTLE TEA AND SYMPATHY,12,2010-12-09 08:34:00,3.95,14243.0,United Kingdom,2010-12-09
19633,537879,22835,HOT WATER BOTTLE I AM SO POORLY,8,2010-12-09 08:34:00,4.65,14243.0,United Kingdom,2010-12-09
19634,537879,85150,LADIES & GENTLEMEN METAL SIGN,6,2010-12-09 08:34:00,2.55,14243.0,United Kingdom,2010-12-09


In [77]:
#Recency we want to focus on the customer and last purchase date

df_uk_recency = df_uk.groupby('CustomerID', as_index= False)['lastdate'].max()
df_uk_recency.columns = ['CustomerID', 'LastPurchaseDate']
df_uk_recency.head()

,CustomerID,LastPurchaseDate
0,12346.0,2011-01-18
1,12747.0,2011-12-07
2,12748.0,2011-12-09
3,12749.0,2011-12-06
4,12820.0,2011-12-06


In [78]:
#We will calculate the recency now

df_uk_recency['Recency'] = df_uk_recency['LastPurchaseDate'].apply(lambda x: (now-x).days)
df_uk_recency.head(3)

,CustomerID,LastPurchaseDate,Recency
0,12346.0,2011-01-18,347
1,12747.0,2011-12-07,24
2,12748.0,2011-12-09,22


We can interpret see the customers last purchase date and as well as how many days ago was the last purchase done. This is recency, now next we'll go to Frequency.

### Frequency (F)

Frequency refers to how many times the customer visited the shop in the given period of time.

In [84]:
df_uk_frequency = df_uk

In [85]:
df_uk_frequency.drop_duplicates(['InvoiceNo', 'CustomerID'],inplace = True)

In [94]:
df_uk_frequency_gb = df_uk_frequency.groupby(by = ['CustomerID'], as_index = False)['InvoiceNo'].count()
df_uk_frequency_gb.columns = ['CustomerID','Frequency']
df_uk_frequency_gb.head()

,CustomerID,Frequency
0,12346.0,1
1,12747.0,10
2,12748.0,196
3,12749.0,5
4,12820.0,4


Now we have the count of unique transactions which represents how many times the customer came to the shop

## Monetary (M)

Monetary (M): How much money the customer spent over time?

In [96]:
# Create a feature called total spent which represents how much the customer spent in the transaction

df_uk['TotalSpent'] = df_uk['UnitPrice']*df_uk['Quantity']
df_uk.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,lastdate,TotalSpent
19632,537879,22114,HOT WATER BOTTLE TEA AND SYMPATHY,12,2010-12-09 08:34:00,3.95,14243.0,United Kingdom,2010-12-09,47.4
19640,537880,21669,BLUE STRIPE CERAMIC DRAWER KNOB,12,2010-12-09 09:14:00,1.25,12963.0,United Kingdom,2010-12-09,15.0


In [97]:
df_uk_monetary = df_uk.groupby('CustomerID', as_index = False).agg({'TotalSpent':'sum'})
df_uk_monetary.columns = ['CustomerID','Monetary']
df_uk_monetary.head()

,CustomerID,Monetary
0,12346.0,77183.60
1,12747.0,658.89
2,12748.0,3739.23
3,12749.0,98.35
4,12820.0,58.20


This is the total amount spent by the customer all the transactions

# RFM Table

We will combine the results from Recency, Frequency, and Monetary table into one table

In [132]:
merge_df = df_uk_recency.merge(df_uk_frequency_gb, on = 'CustomerID')
merge_df.head(3)

,CustomerID,LastPurchaseDate,Recency,Frequency
0,12346.0,2011-01-18,347,1
1,12747.0,2011-12-07,24,10
2,12748.0,2011-12-09,22,196


In [133]:
merge_df.drop(columns = 'LastPurchaseDate', inplace = True) 
#we can drop the LastPurchaseDate column

In [134]:
df_rfm = merge_df.merge(df_uk_monetary, on = 'CustomerID')

In [135]:
df_rfm.head(3)

,CustomerID,Recency,Frequency,Monetary
0,12346.0,347,1,77183.60
1,12747.0,24,10,658.89
2,12748.0,22,196,3739.23


This is our created RFM analysis table, we used customer_id as index and Recency, Frequency, Monetary as our 3 features

Test the accurary of the table

In [136]:
df_uk[df_uk['CustomerID']==15554.0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,lastdate,TotalSpent
110977,545705,20749,ASSORTED COLOUR MINI CASES,2,2011-03-07 09:00:00,7.95,15554.0,United Kingdom,2011-03-07,15.9


In [137]:
df_rfm[df_rfm['CustomerID']==15554.0]

,CustomerID,Recency,Frequency,Monetary
1927,15554.0,299,1,15.9


In [138]:
(now - dt.date(2011,3,7)).days == 299

True

Now, we can confirm that the RFM Table is accurate

In [139]:
#Let's set the index number as customer_id

df_rfm.set_index('CustomerID', inplace = True) #use customer id as index

## Scoring Customers

We need score the customers to figure out the best customer:

We will divide the data into Quartiles and assign scores from 1 to 4 for Recency, Frequency, and Monetary. 4 being the best and 1 being the worst. Final score is combining all the scores for each category.

Best Customers are the ones who score 444, that means 4 in Recency, 4 in Frequency and 4 in Monetary and the worst customer score 111, that means 1 in Recency, 1 in Frequency and 1 in Monetary. 

In [140]:
df_rfm_quant = df_rfm.quantile(q = [0.25,0.50,0.75])
df_rfm_quant

,Recency,Frequency,Monetary
0.25,39.0,1.0,17.4
0.50,71.0,2.0,45.0
0.75,156.0,5.0,121.6


One function for Recency because lower the better and one function for Frequency and Monetary because higher the better 

In [141]:
# Arguments (x = value, p = recency, d = quartiles dict)
def RecScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
# Arguments (x = value, p =  monetary_value, frequency, k = quartiles dict)
def FreMonScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [146]:
df_rfm_segment = df_rfm
df_rfm_segment['RecencyScore'] = df_rfm_segment['Recency'].apply(RecScore, args = ('Recency',df_rfm_quant,))
df_rfm_segment['FrequencyScore'] = df_rfm_segment['Frequency'].apply(FreMonScore, args = ('Frequency',df_rfm_quant,))
df_rfm_segment['MonetaryScore'] = df_rfm_segment['Monetary'].apply(FreMonScore, args = ('Monetary',df_rfm_quant,))
df_rfm_segment.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore
CustomerID,,,,,,
12346.0,347,1,77183.60,1,1,4
12747.0,24,10,658.89,4,4,4
12748.0,22,196,3739.23,4,4,4
12749.0,25,5,98.35,4,3,3
12820.0,25,4,58.20,4,3,3


Combining scores (NOT BY ADDING) but my concatenating.  

In [155]:
df_rfm_segment['RFM_Score'] = df_rfm_segment['RecencyScore'].map(str) + df_rfm_segment['FrequencyScore'].map(str)+df_rfm_segment['MonetaryScore'].map(str)
df_rfm_segment.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score
CustomerID,,,,,,,
12346.0,347,1,77183.60,1,1,4,114
12747.0,24,10,658.89,4,4,4,444
12748.0,22,196,3739.23,4,4,4,444
12749.0,25,5,98.35,4,3,3,433
12820.0,25,4,58.20,4,3,3,433


Like we mentioned before, RFM_Score combines the rankings. 

Best Customers are the ones who score 444, that means 4 in Recency, 4 in Frequency and 4 in Monetary and the worst customer score 111, that means 1 in Recency, 1 in Frequency and 1 in Monetary. While other customers falls somewhere in between.

**Best Customers**

In [161]:
best_cust = df_rfm_segment[df_rfm_segment['RFM_Score']=='444']
best_cust.sample(3)

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score
CustomerID,,,,,,,
12919.0,30,6,267.40,4,4,4,444
17809.0,38,9,1469.04,4,4,4,444
14367.0,30,16,538.95,4,4,4,444


**Loyal Customers**

In [160]:
loyal_cust = df_rfm_segment[df_rfm_segment['FrequencyScore']==4]
loyal_cust.sample(3)

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score
CustomerID,,,,,,,
16746.0,26,18,307.01,4,4,4,444
14735.0,25,32,1309.56,4,4,4,444
15144.0,30,17,499.60,4,4,4,444


**Lost Least Spending Customers**

In [163]:
least_spend = df_rfm_segment[df_rfm_segment['RFM_Score']=='111']
least_spend.sample(2)

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score
CustomerID,,,,,,,
12977.0,178,1,8.5,1,1,1,111
13357.0,279,1,9.9,1,1,1,111


In [169]:
large_spend = df_rfm_segment[df_rfm_segment['MonetaryScore']==4]
lost_cust = df_rfm_segment[df_rfm_segment['RFM_Score']=='144']
almost_lost_cust = df_rfm_segment[df_rfm_segment['RFM_Score']=='244']

Let's Print all values

In [171]:
print('Best Customers:', len(best_cust))
print('Large Spenders:', len(large_spend))
print('Loyal Customers:',len(loyal_cust))
print('Almost Lost Customers:', len(almost_lost_cust))
print('Lost Customers:', len(lost_cust))
print('Lost Least Spending Customers:', len(least_spend))

Best Customers: 356
Large Spenders: 966
Loyal Customers: 752
Almost Lost Customers: 64
Lost Customers: 9
Lost Least Spending Customers: 353


Best Customers should be rewarded with more promotions because it is essential to keep them. We should also keep focus on almost lost customers. Because these are good customers who USED to visit a lot and spend a lot but haven't visited our store in a while.We need to keep these customers and offer them extra initiative to win back their business. 

Loyal Customers and Large spenders are also important for the business. It is important to understand their spending patterns and bundle the most purchase products as a good measure to keep them.

Lost customers are customers who spent a good amount of money but we they are no longer associated with us. We need to understand why they left and make sure our almost lost customers won't become lost customers.We need to see if the competitor is offering better value than us for them to deviate towards them. 

Lost Least Spending Customers: we don't have to worry about them as they barely visit us and even if they do they spend little amount. 

